### URLテーブルを作成

In [1]:
import sqlite3

conn = sqlite3.connect('../data/sqlite3/sqlite3');

# URLテーブルを最初に削除しておく（初期化）
# IF EXISTSでurlテーブルの存在確認
conn.execute('DROP TABLE IF EXISTS url')
# CREATE
conn.execute('CREATE TABLE url(id INTEGER PRIMARY KEY AUTOINCREMENT, url TEXT, docs TEXT)')
conn.close()

### クレンジングとスクレイピングの関数を定義

In [2]:
import re
import unicodedata

from bs4 import BeautifulSoup

# translateで指定する変換法則を指定
translation_table = str.maketrans(dict(zip('()!', '（）!')))

# クレンジング関数
def clense(text):
    text = unicodedata.normalize('NFKC', text).translate(translation_table)
    text = re.sub(r'\s+', '　', text)
    return text

def scrape(html):
    soup = BeautifulSoup(html, 'html.parser')
    for block in soup.find_all(['br', 'p', 'h1', 'h2', 'h3', 'h4']):
        # blockごとに処理
        # 各文章の最後に'<__EOS__>'を挿入
        if len(block.text.strip()) > 0 and block.text.strip()[-1] not in ['。', '!']:
            block.append('<__EOS__>')
    
    # 本文の抽出
    text = '\n'.join([clense(block.text.strip()) 
                      for block in soup.find_all(['p', 'h1', 'h2', 'h3', 'h4']) if len(block.text.strip()) > 0
                     ])
    
    # タイトルの抽出
    # wikipediaの削除
    title = clense(soup.title.text.replace(' - Wikipedia', ''))
    return text, title

### データをsqlite3のurlテーブルにインサート
- glob
- json
- urllib.parse

In [4]:
# 特定のディレクトリに存在する特定のファイルに処理を加えられるライブラリ
import glob
# jsonファイルをあつえるようにするライブラリ
import json
# URLを分割したり解析したりできるライブラリ
import urllib.parse
import sqlite3

conn = sqlite3.connect('../data/sqlite3/sqlite3')
values = []
# 語尾がhtmlのファイルを回して取り出し
for filename in glob.glob('../data/wikipedia/*.html'):
    # utf-8でエンコードしながらfileをfinとして開く
    with open(filename, 'r', encoding='utf-8') as fin:
        html = fin.read()
        # 上記セルの関数を使用
        text, title = scrape(html)
        print('screaped:', title)
        # urllib.parse.quote()はURL 構成要素として使用できるよう特殊文字をクオートしたり非 ASCII 文字を適切にエンコードする
        url = f'https://ja.wikipedia.org/wili/{urllib.parse.quote(title)}'
        print(url)
        sql = "INSERT INTO url (url, docs) VALUES ('" + url + "' , '" + title + "');"
        print(sql)
        conn.execute(sql);
        conn.commit()
        
conn.close()

screaped: ボリビア
https://ja.wikipedia.org/wili/%E3%83%9C%E3%83%AA%E3%83%93%E3%82%A2
INSERT INTO url (url, docs) VALUES ('https://ja.wikipedia.org/wili/%E3%83%9C%E3%83%AA%E3%83%93%E3%82%A2' , 'ボリビア');
screaped: ブルネイ
https://ja.wikipedia.org/wili/%E3%83%96%E3%83%AB%E3%83%8D%E3%82%A4
INSERT INTO url (url, docs) VALUES ('https://ja.wikipedia.org/wili/%E3%83%96%E3%83%AB%E3%83%8D%E3%82%A4' , 'ブルネイ');
screaped: ブークモール
https://ja.wikipedia.org/wili/%E3%83%96%E3%83%BC%E3%82%AF%E3%83%A2%E3%83%BC%E3%83%AB
INSERT INTO url (url, docs) VALUES ('https://ja.wikipedia.org/wili/%E3%83%96%E3%83%BC%E3%82%AF%E3%83%A2%E3%83%BC%E3%83%AB' , 'ブークモール');
screaped: ブータン
https://ja.wikipedia.org/wili/%E3%83%96%E3%83%BC%E3%82%BF%E3%83%B3
INSERT INTO url (url, docs) VALUES ('https://ja.wikipedia.org/wili/%E3%83%96%E3%83%BC%E3%82%BF%E3%83%B3' , 'ブータン');
screaped: チェコ
https://ja.wikipedia.org/wili/%E3%83%81%E3%82%A7%E3%82%B3
INSERT INTO url (url, docs) VALUES ('https://ja.wikipedia.org/wili/%E3%83%81%E3%82%A7%E3%82%B3' 